# **SHORT_TITLE**

In [ ]:
variable = "VARIABLE_NAME" 

In [ ]:
import pandas as pd
import importlib
import re
import warnings
import calendar
import copy
import oceanval
import pickle

import glob
import geopandas as gpd
import nctoolkit as nc
nc.options(parallel=True)
import os
import pickle
import numpy as np
import xarray as xr
from IPython.display import Markdown as md_markdown
from tqdm import tqdm
import hvplot.pandas
from plotnine import *
from IPython.core.interactiveshell import InteractiveShell
from oceanval.tidiers import tidy_info
InteractiveShell.ast_node_interactivity = "all"

try:
    lon_lim = the_lon_lim
    lat_lim = the_lat_lim
except:
    lon_lim = None
    lat_lim = None
if lon_lim is None:
    lon_lim = [-180, 180]
if lat_lim is None:
    lat_lim = [-90, 90]


# in one line
from oceanval.tidiers import fix_unit, df_display, md, md_basic

warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

i_figure = 1
i_table = 1
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")

def bin_value(x, bin_res):
    return np.floor((x + bin_res / 2) / bin_res + 0.5) * bin_res - bin_res / 2



In [ ]:
data_dir = "../../../oceanval_summaries/data"

In [ ]:
data_paths = nc.glob(f"{data_dir}/{variable}/")
i_fig = 1
ff = f"{data_dir}/{variable}/summaries_config.pkl"
with open(ff, "rb") as f:
    summaries = pickle.load(f)
clim_period = f"{summaries[variable].climatology_years[0]}-{summaries[variable].climatology_years[1]}"
robust = summaries[variable].robust
short_name = summaries[variable].short_name

In [ ]:
surface_clim = [x for x in data_paths if "surface_climatology" in x]
if len(surface_clim) > 0:
    md(f"## Sea surface climatology of {short_name}") 
    ds = nc.open_data(surface_clim[0], checks = False)  
    ds.pub_plot(robust = robust)


In [ ]:
if len(surface_clim) > 0:
    md(f"**Figure {i_fig}**: Sea surface climatology of {short_name} ({clim_period}).")
    i_fig += 1

In [ ]:
verticalmean_clim = [x for x in data_paths if "verticalmean_climatology" in x]
if len(verticalmean_clim) > 0:
    md(f"## Vertical mean climatology of {short_name}") 
    ds = nc.open_data(verticalmean_clim[0], checks = False)  
    ds.pub_plot(robust = robust)

In [ ]:
if len(verticalmean_clim) > 0:
    md(f"**Figure {i_fig}**: Climatology of vertical average {short_name} ({clim_period}).")
    i_fig += 1

In [ ]:
verticalintegrated_clim = [x for x in data_paths if "verticalintegrated_climatology" in x]
if len(verticalintegrated_clim) > 0:
    md(f"## Vertical integrated climatology of {short_name}")
    ds = nc.open_data(verticalintegrated_clim[0], checks = False)  
    ds.pub_plot(robust = robust)
    

In [ ]:
if len(verticalintegrated_clim) > 0:
    md(f"**Figure {i_fig}**: Climatology of vertical integrated {short_name} ({clim_period}).")
    i_fig += 1

In [ ]:
spatialmean_timeseries = [x for x in data_paths if "surface_spatialmeantimeseries" in x]
if len(spatialmean_timeseries) > 0:
    md(f"## Spatial mean timeseries of {short_name}") 
    ds = nc.open_data(spatialmean_timeseries[0], checks = False)  
    df = ds.to_dataframe()
    var = ds.variables[0]
    time_name = [x for x in ds.to_xarray().coords if "time" in x][0]
    df = df.rename(columns={var: "value"})  
    # time
    df = df.rename(columns={time_name: "time"})
    df["year"] = [x.year for x in df["time"]] 
    df = df.loc[:,["year", "value"]]
    unit = ds.contents.unit[0]
    unit = fix_unit(unit)
else:
    # empty data frame
    df = pd.DataFrame(columns=["year", "value"])
    unit = ""

In [ ]:
%%capture --no-display
%%R -i df -i short_name -i unit -w 600 -h 400

library(tidyverse)
# element markdown
if (nrow(df) > 0){

df %>%
    ggplot(aes(x = year, y = value)) +
    geom_line() +
    labs(x = "Year", y = paste("Spatial mean of sea surface", short_name, "(", unit, ")")) +
    theme_grey(base_size = 16)+
    theme(axis.title.y = ggtext::element_markdown())
}

In [ ]:
if len(spatialmean_timeseries) > 0:
    md(f"**Figure {i_fig}**: Spatial mean timeseries of {short_name}.")
    i_fig += 1

In [ ]:
spatialsum_timeseries = [x for x in data_paths if "verticalintegrated_spatialsumtimeseries" in x]
if len(spatialsum_timeseries) > 0:
    md(f"## Vertical integrated spatial sum timeseries of {short_name}") 
    ds = nc.open_data(spatialsum_timeseries[0], checks = False)  
    df = ds.to_dataframe()
    var = ds.variables[0]
    time_name = [x for x in ds.to_xarray().coords if "time" in x][0]
    df = df.rename(columns={var: "value"})
    # time
    df = df.rename(columns={time_name: "time"})
    df["year"] = [x.year for x in df["time"]]
    df = df.loc[:,["year", "value"]]
    unit = ds.contents.unit[0]
    unit = fix_unit(unit)
else:
    # empty data frame
    df = pd.DataFrame(columns=["year", "value"])
    unit = ""

In [ ]:
%%capture --no-display
%%R -i df -i short_name -i unit -w 600 -h 400
library(tidyverse)
if (nrow(df) > 0){
# element markdown
df %>%
    ggplot(aes(x = year, y = value)) +
    geom_line() +
    labs(x = "Year", y = paste("Spatial total of vertical integrated", short_name, "(", unit, ")")) +
    theme_grey(base_size = 16)  +
    theme(axis.title.y = ggtext::element_markdown())
}

In [ ]:
if len(verticalintegrated_clim) > 0:
    md(f"**Figure {i_fig}**: Time series of spatial total {short_name}.")
    i_fig += 1

In [ ]:
spatialvertmean_timeseries = [x for x in data_paths if "verticalmean_spatialmeantimeseries" in x]
if len(spatialvertmean_timeseries) > 0:
    md(f"## Vertical mean spatial mean timeseries of {short_name}") 
    ds = nc.open_data(spatialvertmean_timeseries[0], checks = False)  
    df = ds.to_dataframe()
    var = ds.variables[0]
    time_name = [x for x in ds.to_xarray().coords if "time" in x][0]
    df = df.rename(columns={var: "value"})
    # time
    df = df.rename(columns={time_name: "time"})
    df["year"] = [x.year for x in df["time"]]
    df = df.loc[:,["year", "value"]]
    unit = ds.contents.unit[0]
    unit = fix_unit(unit)
else:
    # empty data frame
    df = pd.DataFrame(columns=["year", "value"])


In [ ]:
%%capture --no-display
%%R -i df -i short_name -i unit -w 600 -h 400
library(tidyverse)
if(nrow(df) > 0){ 
# element markdown
df %>%
    ggplot(aes(x = year, y = value)) +
    geom_line() +
    labs(x = "Year", y = paste("Spatial mean of vertical mean", short_name , "(", unit, ")")) +
    theme_grey(base_size = 16)  +
    theme(axis.title.y = ggtext::element_markdown())
}

In [ ]:
if len(spatialvertmean_timeseries) > 0:
    md(f"**Figure {i_fig}**: Time series of spatial mean of vertical mean {short_name}.")
    i_fig += 1